In [1]:
import pandas as pd
import numpy as np
from nsepython import equity_history
from datetime import datetime, timedelta

In [2]:
%run ./yahoo_fetch.ipynb

In [5]:
%run ./technical_analysis_self.ipynb

In [189]:
def get_returns(df):
    for i in [2, 3, 5, 8, 13, 21]:
        df[f'd{i}_high'] = df['high'].shift(-i)
        df[f'd{i}_close'] = df['close'].shift(-i)
        df[f'd{i}_return_pct'] = (df[f'd{i}_close'] - df['next_day_open']) * 100 / df['next_day_open']
        df[f'd{i}_win_flag'] = df[f'd{i}_return_pct'] >= 2
    return df

def get_sliced_df(df, *additional_columns):
#     col_set = ['symbol', 'next_day_open', 'ADX', 'plus_DI', 'minus_DI', 'rsi', 'bb_upper', 'bb_lower'] 
    col_set = ['symbol', 'next_day_open']
    col_set += [i for i in df.columns if '_return_pct' in i or '_flag' in i] + list(additional_columns)
    
    write_df = df[col_set]
    return write_df

In [225]:
aggregate_dict = {}
for i in [2, 3, 5, 8, 13, 21]:
    aggregate_dict[f'd{i}_return_pct'] = ['mean', 'sum', 'max']
    aggregate_dict[f'd{i}_win_flag'] = ['sum']

In [242]:
script = 'infy'
df = get_historical_data(script, '2021-01-01', '2022-10-31').set_index('date')

In [243]:
for fn in [load_volume_sma, load_emas, load_macd, load_bollinger_bands, load_rsi, load_ATR, load_obv]:
    df = fn(df)
    
df = load_ADX(df, 14)
df = load_aroon(df, period=14)
df['next_day_open'] = df['open'].shift(-1)
df = get_returns(df)

#### MA crossovers

In [244]:
def get_crossovers(df, int_1, int_2):
    return df[
        np.where(
            np.logical_and(
                df[f'ema_{int_1}d'] > df[f'ema_{int_2}d'],
                df[f'ema_{int_1}d'].shift(1) < df[f'ema_{int_2}d'].shift(1)
            ),
            True,
            False
        )
    ].copy()

In [245]:
crossover_9_21 = get_crossovers(df, 9, 21)
crossover_9_21 = get_sliced_df(crossover_9_21, 'ema_9d', 'ema_21d')

In [246]:
crossover_9_21.agg(aggregate_dict).T
# , crossover_9_21.shape

,mean,sum,max
d2_return_pct,0.457002,4.570021,4.721266
d2_win_flag,NaN,2.000000,NaN
d3_return_pct,1.112594,11.125945,3.583332
d3_win_flag,NaN,4.000000,NaN
d5_return_pct,0.349334,3.493342,3.841254
d5_win_flag,NaN,3.000000,NaN
d8_return_pct,0.974384,9.743837,3.670412
d8_win_flag,NaN,5.000000,NaN
d13_return_pct,1.843411,16.590703,7.272727
d13_win_flag,NaN,6.000000,NaN


In [247]:
crossover_5_9 = get_crossovers(df, 5, 9)
crossover_5_9 = get_sliced_df(crossover_5_9, 'ema_5d', 'ema_9d')

In [248]:
crossover_5_9.agg(aggregate_dict).T
# , crossover_5_9.shape

,mean,sum,max
d2_return_pct,0.350430,8.410311,5.522420
d2_win_flag,NaN,4.000000,NaN
d3_return_pct,0.712561,17.101462,6.252732
d3_win_flag,NaN,6.000000,NaN
d5_return_pct,0.565145,13.563491,10.023568
d5_win_flag,NaN,7.000000,NaN
d8_return_pct,0.795759,19.098207,7.922839
d8_win_flag,NaN,13.000000,NaN
d13_return_pct,0.992747,23.825926,9.200157
d13_win_flag,NaN,11.000000,NaN


#### MACD crossover

In [249]:
def get_macd_crossover(df):
    return df[
        np.where(
            np.logical_and(df['macd'] > df['macd_9d_signal'], df['macd'].shift(1) < df['macd_9d_signal'].shift(1)),
            True, False
        )
    ].copy()

def get_macd_zero_crossover(df):
    return df[
        np.where(
            np.logical_and(df['macd'] > 0, df['macd'].shift(1) < 0),
            True, False
        )
    ].copy()

In [250]:
get_macd_crossover(df).aggregate(aggregate_dict).T

,mean,sum,max
d2_return_pct,0.048878,0.684296,2.630879
d2_win_flag,NaN,3.000000,NaN
d3_return_pct,0.091910,1.286744,4.893081
d3_win_flag,NaN,3.000000,NaN
d5_return_pct,0.227736,3.188309,7.706829
d5_win_flag,NaN,4.000000,NaN
d8_return_pct,0.317733,4.448267,8.497275
d8_win_flag,NaN,7.000000,NaN
d13_return_pct,1.465017,20.510232,10.449788
d13_win_flag,NaN,8.000000,NaN


In [251]:
get_macd_zero_crossover(df).aggregate(aggregate_dict).T

,mean,sum,max
d2_return_pct,0.378284,4.161124,4.228222
d2_win_flag,NaN,3.000000,NaN
d3_return_pct,0.524202,5.766218,3.885406
d3_win_flag,NaN,2.000000,NaN
d5_return_pct,0.768878,8.457655,6.548753
d5_win_flag,NaN,4.000000,NaN
d8_return_pct,1.688942,16.889424,4.403463
d8_win_flag,NaN,6.000000,NaN
d13_return_pct,1.409107,14.091066,7.272727
d13_win_flag,NaN,4.000000,NaN


In [252]:
get_macd_zero_crossover(df)

,symbol,open,high,low,close,prev_close,volume,pct_change,vol_sma_10d,ema_5d,...,d8_return_pct,d8_win_flag,d13_high,d13_close,d13_return_pct,d13_win_flag,d21_high,d21_close,d21_return_pct,d21_win_flag
date,,,,,,,,,,,,,,,,,,,,,
2021-01-08,INFY,1278.250000,1316.800049,1278.250000,1312.099976,1262.150024,13186251,0.039575,NaN,1288.871274,...,-0.197454,False,1296.449951,1276.199951,-4.910217,False,1332.000000,1305.550049,-2.723339,False
2021-03-04,INFY,1335.000000,1363.849976,1325.900024,1330.349976,1343.550049,12041941,-0.009825,8514786.8,1310.945203,...,4.403463,True,1370.449951,1353.750000,1.900640,False,1434.949951,1430.199951,7.655247,True
2021-05-26,INFY,1367.900024,1400.000000,1362.050049,1397.250000,1361.599976,6850024,0.026182,4977422.4,1365.730751,...,-1.036172,False,1477.000000,1461.800049,4.101987,True,1568.750000,1559.199951,11.038314,True
2021-10-08,INFY,1705.250000,1731.500000,1698.400024,1723.849976,1691.000000,4738376,0.019426,5384966.5,1698.118134,...,4.259811,True,1724.750000,1703.900024,1.302023,False,1750.949951,1740.150024,3.457195,True
2021-10-13,INFY,1695.949951,1713.650024,1680.050049,1709.199951,1685.150024,7104171,0.014272,5783868.7,1697.868318,...,-2.867734,False,1706.449951,1697.900024,-3.198402,False,1800.000000,1787.150024,1.889967,False
2021-11-08,INFY,1730.000000,1741.000000,1703.000000,1737.500000,1708.699951,5245903,0.016855,4444084.1,1714.883781,...,2.573859,True,1718.349976,1691.650024,-2.484506,False,1759.800049,1753.349976,1.072199,False
2021-12-09,INFY,1759.000000,1769.400024,1742.000000,1763.449951,1753.349976,2082171,0.005760,5405768.1,1739.865046,...,2.931817,True,1895.900024,1888.000000,7.272727,True,1836.000000,1814.300049,3.085230,True
2022-03-10,INFY,1844.750000,1844.750000,1810.199951,1825.900024,1813.349976,6547182,0.006921,9906412.2,1785.815542,...,3.219406,True,1919.000000,1903.949951,4.958652,True,1814.000000,1766.550049,-2.615764,False
2022-07-08,INFY,1515.000000,1516.949951,1499.949951,1514.449951,1498.699951,3652582,0.010509,5216523.4,1495.852405,...,1.357863,False,1473.000000,1471.050049,-1.602003,False,1622.000000,1619.400024,8.321072,True


In [253]:
get_macd_crossover(df)

,symbol,open,high,low,close,prev_close,volume,pct_change,vol_sma_10d,ema_5d,...,d8_return_pct,d8_win_flag,d13_high,d13_close,d13_return_pct,d13_win_flag,d21_high,d21_close,d21_return_pct,d21_win_flag
date,,,,,,,,,,,,,,,,,,,,,
2021-01-08,INFY,1278.250000,1316.800049,1278.250000,1312.099976,1262.150024,13186251,0.039575,NaN,1288.871274,...,-0.197454,False,1296.449951,1276.199951,-4.910217,False,1332.000000,1305.550049,-2.723339,False
2021-02-12,INFY,1305.000000,1323.750000,1301.550049,1309.800049,1292.650024,5797875,0.013267,6577572.8,1298.786582,...,-3.462118,False,1347.000000,1343.550049,1.784095,False,1406.000000,1384.000000,4.848485,True
2021-03-03,INFY,1319.199951,1347.000000,1308.000000,1343.550049,1304.500000,9056399,0.029935,7954347.8,1301.242816,...,3.670412,True,1383.250000,1371.550049,2.737831,True,1426.000000,1411.050049,5.696633,True
2021-05-25,INFY,1358.000000,1368.000000,1354.449951,1361.599976,1348.050049,4099340,0.010052,4934518.8,1349.971126,...,1.297609,False,1451.599976,1446.900024,5.775276,True,1525.000000,1503.150024,9.887418,True
2021-08-03,INFY,1647.000000,1658.650024,1635.599976,1655.199951,1631.550049,5513668,0.014495,5423277.7,1627.738317,...,2.582229,True,1753.150024,1738.750000,4.172909,True,1694.300049,1689.099976,1.198251,False
2021-10-18,INFY,1740.000000,1799.000000,1740.000000,1792.150024,1715.750000,13438109,0.044529,7828463.5,1733.269261,...,-6.069458,False,1717.000000,1708.699951,-5.804854,False,1802.000000,1787.449951,-1.463619,False
2021-11-12,INFY,1743.000000,1782.849976,1738.050049,1780.199951,1733.300049,5069009,0.027058,4084055.2,1747.219017,...,-3.349978,False,1750.500000,1748.250000,-1.899443,False,1763.000000,1752.150024,-1.680599,False
2021-12-10,INFY,1760.000000,1763.750000,1746.000000,1759.250000,1763.449951,1823469,-0.002382,5138696.9,1746.326698,...,3.262494,True,1893.800049,1885.550049,6.706092,True,1869.900024,1850.750000,4.736705,True
2022-01-14,INFY,1882.000000,1933.000000,1881.000000,1929.349976,1896.800049,7688506,0.017160,6585944.6,1890.489747,...,-13.409511,False,1779.000000,1738.550049,-10.316989,False,1746.750000,1738.250000,-10.332467,False
